# Document Loading

## Setup dependencies

In [1]:
%pip install -q -U langchain langchain-community langchain-chroma langchain-openai faiss-cpu
%pip install -q -U langchain_experimental lastmile-eval "lastmile-eval[ui]"
%pip install -q -U python-dotenv
%pip install "tracing-auto-instrumentation[langchain]" --upgrade

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lastmile-utils 0.0.24 requires python-dotenv==1.0.0, but you have python-dotenv 1.0.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
  Using cached python_dotenv-1.0.0-py3-none-any.whl.metadata (21 kB)
Using cached python_dotenv-1.0.0-py3-none-any.whl (19 kB)
  Attempting uninstall: python-dotenv
    Found existing installation: python-dotenv 1.0.1
    Uninstalling python-dotenv-1.0.1:
      Successfully uninstalled python-dotenv-1.0.1
Note: you may need to restart the kernel to use updated packages.


## Set Up Environment

In [2]:
import dotenv
dotenv.load_dotenv()

True

## Load documents

In [3]:
from langchain_community.document_loaders import DirectoryLoader 
from langchain_community.document_loaders.text import TextLoader 
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_experimental.text_splitter import SemanticChunker
from langchain.vectorstores import Chroma 
from langchain_openai.embeddings import OpenAIEmbeddings

# from tracing_auto_instrumentation.langchain import LangChainInstrumentor

# # Create an instance of LangChainInstrumentor and instrument the code
# instrumentor = LangChainInstrumentor(project_name="RAG Eval Test")
# instrumentor.instrument()

print("Loading documents...")
knowledgeDirectoryPath = "knowledge/structured"
loader = DirectoryLoader(knowledgeDirectoryPath, glob="**/*.*", loader_cls=TextLoader) 
loaded_docs = loader.load() 

embeddings = OpenAIEmbeddings(model="text-embedding-3-large") 

print("Splitting documents...")
#splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100) 
splitter = SemanticChunker(embeddings)
chunks = splitter.split_documents(loaded_docs) 

print("Creating embeddings...")
vector_store = Chroma.from_documents(chunks, embeddings) 

print("Done loading documents")


Loading documents...
Splitting documents...
Creating embeddings...
Done loading documents


## Query documents

In [4]:
def retrieve_docs(query):
    # Get documents similar to the query ith their scores
    docs_and_score = vector_store.similarity_search_with_score(query, k=5)
    
    # Remove duplicates and unrelated documents
    unique_docs = []
    unique_docs_and_score = []
    seen = set()
    ordered_byScore = sorted(docs_and_score, key=lambda x: x[1])
    for doc in ordered_byScore:
        content = doc[0].page_content
        score = doc[1]
        if score < 1 and content not in seen:
            unique_docs.append(content)
            unique_docs_and_score.append(doc)
            seen.add(content)
            
    context = "\n\n".join(unique_docs)
    
    return context, unique_docs_and_score

## Send query to retriever

In [5]:
context, results = retrieve_docs("I'm a new associate. What should I do to be successful?")
# context, results = retrieve_docs("What is the capital of Argentina?")

results

[(Document(metadata={'source': 'knowledge/structured/Your First 48 Hours - Darnell Self Captions.md'}, page_content="Get around the most successful associates. ## Fast Start Qualification\nWe want to get you paid and promoted, called fast start qualified. The company gives you 20 days to earn your bonuses. Why not do it in your first two days? ## Getting Customers\nTo get fast start qualified, you need three customers and one business partner. One of those customers might become an associate. This increases your belief level. ## Accessing Information\nGain access to information quickly. There are videos that will show you how to launch and structure your business. ## Three-Way Calls\nUse three-way calls to get expert help in answering questions. This way, you learn while your prospect gets the answers they need. ## Continuous Learning\nYou have access to many resources in your back office. Gain that information and education to become more competent and confident. ## Conclusion\nI'm ex

## Send query to LLM

In [10]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, SystemMessage
from lastmile_eval.rag.debugger.tracing import get_lastmile_tracer
from lastmile_eval.rag.debugger.common.types import RagFlowType
from lastmile_eval.rag.debugger.api import LastMileTracer

SCOPED_PROMPT = """
If the answer to the user's question is not contained in the provided context, answer 🤷.
"""
# If the answer to the user question is not contained in the provided context and cannot be inferred from it, 
# answer 🤷.
# """

USE_MARKDOWN_PROMPT = """
Use Markdown to format your response.
"""

TRANSPARENT_CONTEXT = """
Do not mention the context in your answer.
"""

PROJECT_NAME = "RAG Test 4"

# Instantiate LastMile Tracer object
tracer: LastMileTracer = get_lastmile_tracer(
    tracer_name="my-tracer",
    project_name=PROJECT_NAME,
    rag_flow_type=RagFlowType.QUERY,
)

tracer: LastMileTracer = get_lastmile_tracer("My-Project")

def get_prompt(instructions, scoped_answer, use_markdown, context):
    content = instructions \
        + TRANSPARENT_CONTEXT \
        + (SCOPED_PROMPT if scoped_answer else "") \
        + (USE_MARKDOWN_PROMPT if use_markdown else "") \
        + "\nContext:\n" + context 
    prompt = ChatPromptTemplate.from_messages(
        [
            SystemMessage(content=content),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )
    return prompt

@tracer.trace_function()
def get_response(query, modelfamily, model, instructions, scoped_answer, use_markdown, temperature):    
    client = get_client(modelfamily, model, temperature)
    context, docs_with_scores = retrieve_docs(query)
    prompt = get_prompt(instructions, scoped_answer, use_markdown, context)

    chain = prompt | client
    response = chain.invoke({"messages": [HumanMessage(content=query)]})
    
    metadata = get_metadata(modelfamily, model, response)
    
    # Log query event to the trace
    tracer.add_query_event(
        query=query,
        llm_output=response.content,
        system_prompt="system prompt",
        metadata={"llm_name": model, "temperature": temperature, "model_family": modelfamily},
    )
    
    return {
        "content": response.content,
        "docs_with_scores": docs_with_scores,
        "metadata": metadata,
        "prompt": prompt,
        }
    
def get_client(model_family, model, temperature):
    if model_family == "openai":
        from langchain_openai import ChatOpenAI
        model = ChatOpenAI(model=model, temperature=temperature)
    elif model_family == "anthropic":
        from langchain_anthropic import ChatAnthropic
        model = ChatAnthropic(model=model, temperature=temperature)
    else:
        raise ValueError(f"Model {model_family} not recognized")
    
    return model

def get_metadata(model_family, model, response):
    if model_family == "anthropic":
        usage = {k: v for k, v in response.response_metadata["usage"].items() if k in ("input_tokens", "output_tokens")}
    elif model_family == "openai":
        usage = {k: v for k, v in response.usage_metadata.items() if k in ("input_tokens", "output_tokens")}
    else:
        raise ValueError(f"Model family {model_family} not recognized")
    
    total_tokens = usage["input_tokens"] + usage["output_tokens"]

    return {"model": model, **usage, "total_tokens": total_tokens}


[WARNING] 2024-07-10 20:59:09,297 __init__.py:518: Overriding of current TracerProvider is not allowed
[DEBUG] 2024-07-10 20:59:09,299 connectionpool.py:1051: Starting new HTTPS connection (1): lastmileai.dev:443
[DEBUG] 2024-07-10 20:59:09,299 connectionpool.py:1051: Starting new HTTPS connection (1): lastmileai.dev:443
[DEBUG] 2024-07-10 20:59:09,672 connectionpool.py:546: https://lastmileai.dev:443 "GET /api/evaluation_projects/list?name=RAG+Test+4 HTTP/11" 200 344
[WARNING] 2024-07-10 20:59:09,700 __init__.py:518: Overriding of current TracerProvider is not allowed


In [11]:
get_response(
    "I'm a new associate. What should I do to be successful?",                  # query
    "openai",                                                                   # Model family
    "gpt-3.5-turbo",                                                            # Model
    "",                                                                         # Instructions       
    True,                                                                       # Scoped answer    
    True,                                                                       # Use markdown
    0                                                                           # Temperature  
    )

[DEBUG] 2024-07-10 20:59:09,709 _config.py:80: load_ssl_context verify=True cert=None trust_env=True http2=False
[DEBUG] 2024-07-10 20:59:09,710 _config.py:146: load_verify_locations cafile='/Users/raggif/python/streamit/hello_streamlit/.venv/lib/python3.11/site-packages/certifi/cacert.pem'
[DEBUG] 2024-07-10 20:59:09,762 _config.py:80: load_ssl_context verify=True cert=None trust_env=True http2=False
[DEBUG] 2024-07-10 20:59:09,763 _config.py:146: load_verify_locations cafile='/Users/raggif/python/streamit/hello_streamlit/.venv/lib/python3.11/site-packages/certifi/cacert.pem'
[DEBUG] 2024-07-10 20:59:09,777 _base_client.py:447: Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'post_parser': <function Embeddings.create.<locals>.parser at 0x138c67a60>, 'json_data': {'input': [[40, 2846, 264, 502, 22712, 13, 3639, 1288, 358, 656, 311, 387, 6992, 30]], 'model': 'text-embedding-3-large', 'encoding_format': 'base64'}}
[DEBUG] 2024-07-10 20:59:09,779 _base_client.py:9

{'content': "To be successful as a new associate, here are some key steps you can take:\n\n1. **Fast Start Qualification**: Aim to become fast start qualified within the first two days by getting three customers and one business partner. This will boost your belief level and set you on the path to success.\n\n2. **Getting Customers**: Focus on acquiring customers and building relationships. One of your customers might even transition into becoming an associate, further enhancing your success.\n\n3. **Accessing Information**: Make use of the available resources like videos to quickly learn how to launch and structure your business effectively.\n\n4. **Networking**: Surround yourself with successful associates. Attend events, whether in person or via Zoom, to learn from the best and gain valuable insights.\n\n5. **Continuous Learning**: Take advantage of the resources in your back office for continuous learning. The more informed and educated you are, the more competent and confident you